In [1]:
import pandas as pd
from datetime import datetime, timedelta


In [2]:
train = pd.read_csv('train.csv')
attendance = pd.read_csv('attendance.csv')
submission = pd.read_csv('submission.csv')


In [3]:

# ตั้ง format เวลาให้ถูกต้อง
#train['datetime'] = pd.to_datetime(train['datetime'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')
attendance['datetime'] = pd.to_datetime(attendance['datetime'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')


In [4]:

# ลบข้อมูลที่มีค่าเวลาไม่ถูกต้อง
#train.dropna(subset=['datetime'], inplace=True)
attendance.dropna(subset=['datetime'], inplace=True)


In [5]:

# เพิ่ม column 'house' ใน submission โดยดูจาก attendance
submission = pd.merge(submission, attendance[['user_id', 'datetime']], on='user_id')



In [6]:
submission

user_id   house                   datetime
0     6e009a96-2b8b-4048-8b0c-20a7924a4b4f  house3 2024-04-19 17:30:14.433098
1     6e009a96-2b8b-4048-8b0c-20a7924a4b4f  house3 2024-04-20 08:39:08.013977
2     6e009a96-2b8b-4048-8b0c-20a7924a4b4f  house3 2024-04-20 12:48:46.776276
3     6e009a96-2b8b-4048-8b0c-20a7924a4b4f  house3 2024-04-20 17:38:35.927190
4     6e009a96-2b8b-4048-8b0c-20a7924a4b4f  house3 2024-04-21 08:36:59.624088
...                                    ...     ...                        ...
4256  7474c4b5-d72d-4213-ad48-c8376a66cf35     NaN 2024-05-15 17:43:44.548400
4257  7474c4b5-d72d-4213-ad48-c8376a66cf35     NaN 2024-05-16 08:43:56.181531
4258  7474c4b5-d72d-4213-ad48-c8376a66cf35     NaN 2024-05-16 12:51:12.472693
4259  7474c4b5-d72d-4213-ad48-c8376a66cf35     NaN 2024-05-16 17:30:30.331964
4260  7474c4b5-d72d-4213-ad48-c8376a66cf35     NaN 2024-05-17 13:52:46.018001

[4261 rows x 3 columns]

In [7]:
# จัดกลุ่มข้อมูลโดยใช้เวลาเช็คชื่อและบ้าน
submission['hour'] = submission['datetime'].dt.hour
grouped_data = submission.groupby(['house', 'hour']).size().reset_index(name='count')

# เอาเอาแค่บุคคลที่มีเวลาใกล้เคียงกันอยู่บ้านหลังเดียวกัน และเช็คชื่ออยู่ในนาทีเดียวกัน
threshold_time = timedelta(minutes=1)
current_group = None
result = []

for _, row in grouped_data.sort_values(['house', 'hour']).iterrows():
    if current_group is None or (row['house'] != current_group['house'] or row['hour'] != current_group['hour']):
        current_group = {'house': row['house'], 'hour': row['hour'], 'count': 0}
    if abs(row['hour'] - current_group['hour']) <= 1 and row['count'] == current_group['count']:
        result.append((row['house'], row['hour']))
        current_group = None
    else:
        current_group['count'] += 1

# เขียนข้อมูลกลุ่มที่เป็นไปตามเงื่อนไขเข้าไฟล์ output csv
with open('output_groups.csv', 'w') as f:
    for group in result:
        f.write(f"{group[0]},{group[1]}\n")